In [ ]:
import ipywidgets as widgets
from pathlib import Path

import random
from poker import Card, Suit, Rank

import pandas as pd
import numpy as np


In [ ]:
class Player(object):
    
    def __init__(self):
        self.hand = [] 
        self.numCards = 2
        self.calls = []
        self.calledBluff = False
        self.playerAI = False
        
    def makeList(self):
        allPlayers = {}
        numPl = int(input('Enter number of players: '))
        
        if numPl >= 2 and numPl < 7:
            for e in range(numPl):
                name = "Player" + str(e+1)
                allPlayers[name] = {'Hand':Player().hand , 
                                    'Calls':Player().calls,
                                    'Number of Cards' : Player().numCards, 
                                    'Called Bluff?' : Player().calledBluff, 
                                    'AI Player': Player().playerAI}
            return allPlayers
        else:
            print("Invalid number of players. Enter a number between 2 and 6.")
            return Player().makeList()
        
    def detAI(players,player):
        #set player names or set into AI mode
        setAI = input("Is " + player + " an AI Player? 'Y' or 'N':").upper()
        if setAI == 'Y':
            players[player]['AI Player'] = True
        elif setAI == 'N':
            pass
        else: 
            print('Input not valid. Try again.')
            return Player.detAI(players,player)
    
    def dispHands(roundNum, players): #Card Display
        tab_contents = ['']
        #make more tabs to # of players
        for e in range(len(players)):   
            tab_contents.append('Hand:')

        children = [widgets.Text(description=name) for name in tab_contents]
        tab = widgets.Tab()
        tab.children = children
        tab.set_title(0,roundNum)

        for i in range(len(children)):
            tab.set_title(i+1, 'Player '+str(i+1))
        tab.selected_index = 0

        #fill Tab with Cards
        count = 1
        for hand in players:
            tab.children[count].value = str(players[hand]['Hand'][0])+ ", " +str(players[hand]['Hand'][1])
            count += 1

        return tab

# Save to a DataFrame then onto a CSV

In [ ]:
class Save:
    
    df = pd.DataFrame()
    breakLine = pd.Series(name='Round finished. ', dtype=object)
    folder = 'game_data/'
     
    def store(round_data): #call when round is over
        if Save.df.empty:
            Save.df = pd.DataFrame(round_data)
            Save.df.drop(["Number of Cards"], inplace = True)
            Save.df = Save.df.transpose()
            
            Save.df = Save.df.append(Save.breakLine)
            Save.df=Save.df.replace(np.NaN, '--')
        else:
            df2 = pd.DataFrame(round_data)
            df2.drop(["Number of Cards"], inplace = True)
            df2 = df2.transpose()
            
            Save.df = Save.df.append(df2)
            Save.df = Save.df.append(Save.breakLine)
            Save.df = Save.df.replace(np.NaN, '--')
            
        display(Save.df)

    def toCSV(): #call when game is over
        name, count, csv = 'game',0,'.csv'
        file_name = name + str(count) + csv
        while Path(Save.folder + file_name).exists():
            count += 1
            file_name = name + str(count) + csv
        Save.df.to_csv(Save.folder + file_name)
        
    def readCSV(file_name): #call to read previous game data
        read = pd.read_csv(Save.folder + file_name)
        read = read.rename(columns={'Unnamed: 0' : 'Players'})
        #read = read.set_index('Players')
        return read
    

# Helper Functions

In [ ]:
class Helper:
    
    def getSuits(self):
        handSuits = []
        for card in self:
            handSuits.append(card.suit)
        return handSuits

    def getRanks(self):
        handRanks = []
        for card in self:
            if card.rank == Rank('T'):
                handRanks.append(10)
            elif card.rank == Rank('J'):
                handRanks.append(11)
            elif card.rank == Rank('Q'):
                handRanks.append(12)
            elif card.rank == Rank('K'):
                handRanks.append(13)
            elif card.rank == Rank('A'):
                handRanks.append(14)
            else:
                handRanks.append(int(str(card.rank)))
        return handRanks
    
    def setSuits(self):
        print('\nSPADES = s, HEARTS = h, DIAMONDS = d, CLUBS = c\n')
        suitInput = input('Suit: ')
        if suitInput != 's' or suitInput != 'h' or suitInput != 'd' or suitInput != 'c':
            print('Input not valid. Enter "s", "h", "d", or "c".')
            return Helper().setSuits()
        else:
            return Suit(suitInput)
    
    def setRanks(self):
        print('\n3, 4, 5, 6, 7, 8 , 9, T, J, Q, K, A\n')
        rankInput = input('Rank: ').upper()
        
        if rankInput.isnumeric(): 
            rankInput = int(rankInput)
            if rankInput < 3 or rankInput > 9:
                if rankInput == 2:
                    print('2 is a wildcard and not a valid rank. Enter a different rank.')
                    return Helper().setRanks() 
                else:
                    print('Rank not valid. Enter a different rank.')
                    return Helper().setRanks()
            else: 
                pass
        elif rankInput == 'T':
            rankInput = 10
        elif rankInput == 'J':
            rankInput = 11
        elif rankInput == 'Q':
            rankInput = 12
        elif rankInput == 'K':
            rankInput = 13
        elif rankInput == 'A':
            rankInput = 14
        else:
            print('Rank not valid. Enter a different rank.')
            return Helper().setRanks()
        
        return rankInput
    
    def setHand(self, players):
        calledHand = [None]
        
        display(hand)
        print("Or enter '0' to call bluff.\n")
        handInput = int(input('Call: '))
        
        if handInput > 9:
            print('Input not valid. Enter a number between 0 and 9.')
            return Helper().setHand(players)
        elif handInput == 0:
            print('Bluff called. Combine cards...')
        else:
            calledHand.insert(0, handInput)
            if handInput == 3 or handInput == 7:
                inputR = Helper().setRanks()
                calledHand.insert(2,inputR)
                print('Enter 2nd rank.')
                inputR2 = Helper().setRanks()
                calledHand.insert(3,inputR2)
            elif handInput == 6:
                inputS = Helper().setSuits()
                calledHand.insert(1, inputS)
            elif handInput == 9:
                inputS = Helper().setSuits()
                calledHand.insert(1, inputS)
                inputR = Helper().setRanks()
                calledHand.insert(2, inputR)
            else:
                inputR = Helper().setRanks()
                calledHand.insert(2, inputR)
        return calledHand

    def numOfSameCard(self):
        ranks = Helper.getRanks(self)
        sameRank = max(ranks.count(2),ranks.count(3),ranks.count(4),ranks.count(5),
                     ranks.count(6),ranks.count(7),ranks.count(8),ranks.count(9),
                     ranks.count(10),ranks.count(11),ranks.count(12),ranks.count(13),
                     ranks.count(14))
        return sameRank
    
    def removeDup(self):
        rd = list(dict.fromkeys(self))
        rd.sort()
        return rd
    
    def stringCall(self, call):
        
        if call[0] == 3 or call[0] == 7:
            str_hand = hand[call[0]]
            str_rank = str(Rank(call[2]))
            str_rank2 = str(Rank(call[3]))
            return str_hand + ' of  ' + str_rank + ' & ' + str_rank2
        elif call[0] == 5:
            str_hand = hand[call[0]]
            str_rank = str(Rank(call[2]))
            return str_hand + ' to ' + str_rank
        elif call[0] == 6:
            str_hand = hand[call[0]]
            str_suit = str(Suit(call[1]))
            return str_hand + ' of ' + str_suit
        elif call[0] == 9:
            str_hand = hand[call[0]]
            str_suit = str(Suit(call[1]))
            str_rank = str(Rank(call[2]))
            return str_hand + ' of ' + str_suit + ' to ' + str_rank
        else:
            str_hand = hand[call[0]]
            str_rank = str(Rank(call[2]))
            return str_hand + ' of ' + str_rank
    

# Game Functions

In [ ]:
class Game:
    
    global deck, suit, rank, hand
    deck = list(Card)
    suit = list(Suit)
    rank = list(Rank)
    hand = {1:'High Card',2:'Pair',3:'Two Pair',4:'Three of a Kind',5:'Straight',6:'Flush',7:'Full House',8:'Four of a Kind',9:'Straight Flush'}
    prevCall = [None]
    pool_length = 0
    
    def recall():
        deck = list(Card)
    
    def shuffleDeck():
        random.shuffle(deck)

    def deal():
        return deck.pop(0)
    
    def dealToAllPlayers(players):
        count = 1
        for player in players:
            for e in range(players[player]['Number of Cards']):
                if players[player]['Number of Cards'] != 6:
                    players[player]['Hand'].append(Game.deal())
            count += 1
        pool_length = count
            
    def combineCards(players):
        allCards = []
        for e in players:
            allCards = allCards + players[e]['Hand']
        return allCards
    
    def handCall(players):
        currCall = Helper().setHand(players)
        if currCall == [None]:
            if Game.prevCall == [None]:
                print('\nFirst caller cannot call bluff. Make a hand call.')
                return Helper().setHand(players)
            else:
                return Game.bluffCall(players)
        else:
            pass
        
        if Game.prevCall == [None]:
            Game.prevCall = currCall
            return currCall
         
        elif currCall[0] > Game.prevCall[0]: 
            Game.prevCall = currCall
            return currCall
        
        elif currCall[0] == Game.prevCall[0]:
            if currCall[1] == None:
                if currCall[2] > Game.prevCall[2]:
                    Game.prevCall = currCall
                    return currCall
                else:
                    print('\nCall needs to be higher than the previous call. The previous call was:')
                    display(Helper().stringCall(Game.prevCall))
                    return Game.handCall(players)
            else: 
                if currCall[1] > Game.prevCall[1]:
                    Game.prevCall = currCall 
                    return currCall
                else: 
                    print('\nCall needs to be higher than the previous call. The previous call was:')
                    display(Helper().stringCall(Game.prevCall))
                    return Game.handCall(players)
        
        elif currCall[0] < Game.prevCall[0]: 
            print('\nCall needs to be higher than the previous call. The previous call was:')
            display(Helper().stringCall(Game.prevCall))
            return Game.handCall(players)

    def bluffCall(players):
        pool = Game.combineCards(players)
        
        if Game.prevCall[0] == 1:
            check = CheckHands.checkHigh(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 2:  
            check = CheckHands.checkPair(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 3:
            check = CheckHands.check2Pair(pool, Game.prevCall[2],Game.prevCall[3])
        elif Game.prevCall[0] == 4:
            check = CheckHands.check3Kind(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 5:
            check = CheckHands.checkStraight(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 6:
            check = CheckHands.checkFlush(pool, Game.prevCall[1])
        elif Game.prevCall[0] == 7:
            check = CheckHands.checkFullHouse(pool, Game.prevCall[2], Game.prevCall[3])
        elif Game.prevCall[0] == 8:
            check = CheckHands.check4Kind(pool, Game.prevCall[2])
        elif Game.prevCall[0] == 9:
            check = CheckHands.checkStraightFlush(pool, Game.prevCall[1], Game.prevCall[2])
        
        if check == False:
            print('\nBluff call is accurate. Hand caller gets an additional card.')
            return False
        else:
            print('\nBluff call is not accurate. Bluff caller gets an additional card.')
            return True
        
    def startRound(players,round_over):
        prevPlayer = ''
        
#         #display round info
#         display(Player.dispHands(round_num, players))
#         round_num += 1

        while not round_over:
            for player in players:
                if players[player]['AI Player'] == True:
                    #call AI Player function
                    #function should return either a list or boolean
                    display(player + ': AI')
                    call = AIone.handCall(players,players[player]['Hand'],Game.prevCall)
                else:
                    #get hand calls
                    display(player)
                    print('Hand: ' + str(players[player]['Hand']))
                    call = Game.handCall(players)
                
                #check if player called hand or called bluff
                if type(call) == list:
                    players[player]['Calls'].append(Helper().stringCall(call))
                    print('Call: ' + Helper().stringCall(call))
                    prevPlayer = player
                else:
                    players[player]['Called Bluff?'] = True
                    if call == True: 
                        #add card to bluff caller
                        players[player]['Number of Cards'] += 1
                        if players[player]['Number of Cards'] == 6:
                            del players[player]
                    else:
                        #add card to hand caller instead
                        players[prevPlayer]['Number of Cards'] += 1
                        if players[prevPlayer]['Number of Cards'] == 6:
                            del players[prevPlayer]          
                    #round over, append round data into Game data file.             
                    print('Round finished.')
                    Save.store(players)
                    round_over = True
                    break
                    
    def roundReset(self,players):  
        for player in players:
            players[player]['Hand'] = []
            players[player]['Calls'] = []
            players[player]['Called Bluff?'] = False
        deck = list(Card)
    
    def run():
        #set number of players and create list 
        players = Player().makeList()
        for player in players: 
            Player.detAI(players,player)
        
        #start game
        game_over = False
        round_over = False
        round_count = 1
        round_num = 'Round ' + str(round_count)
        while not game_over:
            if len(players) != 1:
                #round display
                print('\nRound ' + str(round_count) + ':')
                round_count += 1
                
                #reset info before starting a new round
                Game.prevCall = [None]
                Game().roundReset(players)
                
                #shuffle deck and distribute
                Game.recall()
                Game.shuffleDeck()
                Game.dealToAllPlayers(players)
                
                Game.startRound(players,round_over)
            else:
                print('Game over.')
                print(list(players.keys())[0] + 'won!')
                Save.toCSV()
                game_over = True


# Check Poker Hands

In [ ]:
class CheckHands:
    rank = []
    
    def removeTwo(self):   #Used for the straight function Mayber
        for card in self:
            if card == 2:
                pos = self.index(card)
                self.pop(pos)
        return self
    
    def countTwo(self):
        count = 0
        for card in self:
            if card == 2:
                count += 1
        return count
    
    def checkHigh(self, inpRank):
        self = Helper.getRanks(self)
        
        if self.count(inpRank) > 0:
            return True
        else:
            if CheckHands.countTwo(self) > 0:
                return True
            else:
                return False
    
    def checkFlush(self, inpSuit):
        hand = self
        self = Helper.getSuits(self)
        check = Suit(inpSuit)
        
        s = suit[3]
        h = suit[2]
        d = suit[1]
        c = suit[0]
        num = max(self.count(s),self.count(h),self.count(d),self.count(c))
        if num == self.count(s):
            maxSuit = s
        elif num == self.count(h):
            maxSuit = h
        elif num == self.count(d):
            maxSuit = d
        elif num == self.count(c):
            maxSuit = c
        if (num < 5) or check != maxSuit:
            numTwo = CheckHands.countTwo(self)
            for card in hand:
                if '2' + inpSuit == str(card):
                    numTwo = numTwo - 1
            if numTwo + num < 5:
                return False, maxSuit
            else:
                return True, maxSuit
        else:
            return True, maxSuit
     
    def checkStraight(self, high):
        self = Helper.getRanks(self) #remove duplicates and sort
        numTwo = CheckHands.countTwo(self)
        self = CheckHands.removeTwo(self)
        if high in self:
            straight = list(range(int(high - 4),high+1))
            count = 0
            inARow = 0
            for num in straight:
                if num in self:
                    inARow += 1
                elif numTwo > 0:
                    numTwo -= 1
                    inARow += 1
                count += 1
            if inARow < 5:
                return False
            else:
                return True
        else:
            return False
        
    def checkStraightFlush(self, suit, high):
        isFlush, maxSuit = CheckHands.checkFlush(self, suit)
        if isFlush:
            flushRanks = []
            i = 0
            for e in Helper.getSuits(self):
                if e == maxSuit or Helper.getRanks(self)[i] == 2:
                    flushRanks.append(self[i])
                i += 1
            if CheckHands.checkStraight(flushRanks, high):
                return True
            else:
                return False
        else:
            return False
        
    def check4Kind(self, inpQuad):
        ranks = Helper.getRanks(self)
        if ranks.count(inpQuad) == 4:
            return True
        else:
            if(ranks.count(inpQuad) + CheckHands.countTwo(self)) == 4:
                diff = abs(ranks.count(inpQuad) - 4)
                for e in range(diff):
                    ranks.remove(2)
                return True
            else:
                return False
        
    def check3Kind(self, inpTriple):
        ranks = Helper.getRanks(self)
        if ranks.count(inpTriple) >= 3:

            return True
        else:
            if(ranks.count(inpTriple) + CheckHands.countTwo(self)) >= 3:
                diff = abs(ranks.count(inpTriple) - 3)
                for e in range(diff):
                    ranks.remove(2)
                return True
            else:
                return False
    
    def checkPair(self, inpDouble):
        ranks = Helper.getRanks(self)
        if ranks.count(inpDouble) >=2:
            return True
        else:
            if(ranks.count(inpDouble) + CheckHands.countTwo(self)) >= 2:
                diff = abs(ranks.count(inpDouble) - 2)
                for e in range(diff):
                    ranks.remove(2)
                return True
            else:
                return False
        
    def checkFullHouse(self, inpTriple, inpDouble):
        if inpTriple == inpDouble:
            return False
        if CheckHands.check3Kind(self, inpTriple) and CheckHands.checkPair(self, inpDouble):
            return True
        else:
            return False
    
    def check2Pair(self, inpDouble, inpDoubleTwo):
        if inpDouble == inpDoubleTwo:
            return False
        if CheckHands.checkPair(self, inpDouble) and CheckHands.checkPair(self, inpDoubleTwo):
            return True
        else:
            return False

# Get Probabilities

In [ ]:
class Heuristic:
    
    def getRandomHand(numCards, myHand):
        sampleDeck = list(Card)
        random.shuffle(sampleDeck)
        cards = []
        #remove cards in deck that are in your hand
        if myHand == []:
            pass
        else:
            for card in myHand:
                loc = sampleDeck.index(card)
                cards.append(sampleDeck.pop(loc))
        
        for num in range(numCards - len(myHand)):
            currCard = sampleDeck.pop(0)
            cards.append(currCard)
        return cards
    
    def probGeneral(numCards, trials, myHand):
        numStraightFlush = 0
        numFourKind = 0
        numFullHouse = 0
        numFlush = 0
        numStraight = 0
        numThreeKind = 0
        numTwoPair = 0
        numPair = 0
        
            
        a, numPair = Heuristic.calcPair(myHand, numCards)
        b, c,  numTwoPair = Heuristic.calc2Pair(myHand, numCards)
        d, numThreeKind = Heuristic.calc3Kind(myHand, numCards)
        e, numStraight = Heuristic.calcStraight(myHand, numCards)
        f, numFlush = Heuristic.calcFlush(myHand, numCards)
        g, h, numFullHouse = Heuristic.calcFullHouse(myHand, numCards)
        i, numFourKind = Heuristic.calc4Kind(myHand, numCards)
        j, k, numStraightFlush = Heuristic.calcStraightFlush(myHand, numCards)
        
        return 1, numPair/trials, numTwoPair/trials, numThreeKind/trials, numStraight/trials, \
                   numFlush/trials, numFullHouse/trials, numFourKind/trials, numStraightFlush/trials
        
    def calcHigh(myHand):
        ranks = Helper.getRanks(myHand)
        maxRank = max(ranks)
        return maxRank
    
    def calcPair(myHand, numCards):
        trials = 1000
        numCounter = {2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for num in range(3,15):
                if CheckHands.checkPair(handTrial, num):
                    numCounter[num] += 1
        return (max(numCounter, key = numCounter.get)), numCounter[max(numCounter, key = numCounter.get)]
    
    def calc2Pair(myHand, numCards):
        firstNum, temp = Heuristic.calcPair(myHand, numCards)
        trials = 1000
        numCounter = {2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for num in range(3,15):
                if CheckHands.check2Pair(handTrial, firstNum, num):
                    numCounter[num] += 1
        return firstNum, (max(numCounter, key = numCounter.get)), numCounter[max(numCounter, key = numCounter.get)]
    
    def calc3Kind(myHand, numCards):
        trials = 1000
        numCounter = {2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for num in range(3,15):
                if CheckHands.check3Kind(handTrial, num):
                    numCounter[num] += 1
        return (max(numCounter, key = numCounter.get)), numCounter[(max(numCounter, key = numCounter.get))]
    
    def calcStraight(myHand, numCards):
        trials = 1000
        numCounter = {2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for num in range(6,15):
                if CheckHands.checkStraight(handTrial, num):
                    numCounter[num] += 1
        return max(numCounter, key = numCounter.get), numCounter[(max(numCounter, key = numCounter.get))]
    
    def calcFlush(myHand, numCards):
        trials = 1000
        suitCounter = {Suit('♣'):0, Suit('♦'):0, Suit('♥'):0, Suit('♠'):0 }
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for suit in list(Suit):
                isFlush, maxSuit = CheckHands.checkFlush(handTrial, str(suit))
                if isFlush and maxSuit == suit:
                    suitCounter[suit] += 1
        return max(suitCounter, key = suitCounter.get), suitCounter[max(suitCounter, key = suitCounter.get)]
    
    def calcFullHouse(myHand, numCards):
        firstNum, temp = Heuristic.calc3Kind(myHand, numCards)
        trials = 1000
        numCounter = {2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for num in range(3,15):
                if CheckHands.checkFullHouse(handTrial, firstNum, num):
                    numCounter[num] += 1
        return firstNum, (max(numCounter, key = numCounter.get)), numCounter[(max(numCounter, key = numCounter.get))]
    
    def calc4Kind(myHand, numCards):
        trials = 1000
        numCounter = {2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for num in range(3,15):
                if CheckHands.check4Kind(handTrial, num):
                    numCounter[num] += 1
        return (max(numCounter, key = numCounter.get)), numCounter[(max(numCounter, key = numCounter.get))]
    
    def calcStraightFlush(myHand, numCards):
        suitBest, temp = (Heuristic.calcFlush(myHand, numCards))
        suitBest = str(suitBest)
        trials = 1000
        numCounter = {2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 11:0, 12:0, 13:0, 14:0}
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            for num in range(6,15):
                if CheckHands.checkStraightFlush(handTrial, suitBest, num):
                    numCounter[num] += 1
        return suitBest, (max(numCounter, key = numCounter.get)), numCounter[(max(numCounter, key = numCounter.get))]

# AI

In [ ]:
class AIone:
    
    probHigh, probPair, prob2Pair, prob3Kind, probStraight, probFlush, probFullHouse,\
    prob4Kind, probStraightFlush = Heuristic.probGeneral(numCards, 1000, myHand)

    generalProb = {probHigh: 1, probPair: 2, prob2Pair: 3, prob3Kind: 4, probStraight: 5, probFlush: 6,\
    probFullHouse: 7, prob4Kind: 8, probStraightFlush: 9}
        
    def bluff(myHand, HandGuess, rankGuess, rankGuessTwo, suitGuess, numCards): #true false output on whether AI should bluff
        numMatch = 0
        bluffConstant = 0.25
        trials = 2000
        for trial in range(trials):
            handTrial = Heuristic.getRandomHand(numCards, myHand)
            if int(HandGuess) == 1:
                if CheckHands.checkHigh(handTrial, str(rankGuess)):
                    numMatch += 1
            elif int(HandGuess) == 2:
                if CheckHands.checkPair(handTrial, str(rankGuess)):
                    numMatch += 1
            elif int(HandGuess) == 3:
                if CheckHands.check2Pair(handTrial, str(rankGuess), str(rankGuessTwo)):
                    numMatch += 1
            elif int(HandGuess) == 4:
                if CheckHands.check3Kind(handTrial, str(rankGuess)):
                    numMatch += 1
            elif int(HandGuess) == 5:
                if CheckHands.checkStraight(handTrial, (rankGuess)):
                    numMatch += 1
            elif int(HandGuess) == 6:
                if CheckHands.checkFlush(handTrial, suitGuess):
                    numMatch += 1
            elif int(HandGuess) == 7:
                if CheckHands.checkFullHouse(handTrial, str(rankGuess), str(rankGuessTwo)):
                    numMatch += 1
            elif int(HandGuess) == 8:
                if CheckHands.check4Kind(handTrial, str(rankGuess)):
                    numMatch += 1
            elif int(HandGuess) == 9:
                if CheckHands.checkStraightFlush(handTrial, suitGuess, (rankGuess)):
                    numMatch += 1
        if numMatch/trials > bluffConstant:
            return numMatch/trials
        else:
            return numMatch/trials
        
    def guess(myHand, prevHand, prevRank, prevRankTwo, prevSuit, numCards):  #what the AI will guess given that round
        
        sameHand = False
        probList = []
        for prob in generalProb:
            probList.append(prob)
        while True:
            maxChance = max(probList)
            if prevHand == None:
                currRank = Heuristic.calcHigh(myHand)
                return [1, None, currRank, None]
            if prevRank == None:
                prevRank = 0
            if prevRankTwo == None:
                prevRankTwo = 0
            if prevSuit == None:
                prevSuit = Suit('♣')
                
            elif generalProb[maxChance] < prevHand:
                probList.remove(maxChance)
                
            elif generalProb[maxChance] == prevHand:  #check when they are the same hand level
                if generalProb[maxChance] == 1:
                    currRank, temp = Heuristic.calcHigh(myHand)
                    if prevRank >= currRank:
                        probList.remove(maxChance)
                    else:
                        return [1, None, currRank, None]
                if generalProb[maxChance] == 2:
                    currRank, temp = Heuristic.calcPair(myHand, numCards)
                    if prevRank >= currRank:
                        probList.remove(maxChance)
                    else:
                        return [2, None, currRank, None]
                if generalProb[maxChance] == 3:
                    currRank, currRankTwo, temp = Heuristic.calc2Pair(myHand, numCards)
                    if prevRank >= currRank:
                        if prevRankTwo >= currRankTwo:
                            probList.remove(maxChance)
                        else:
                            return [3, None, currRank, currRankTwo]
                    else:
                        return [3, None, currRank, currRankTwo]
                if generalProb[maxChance] == 4:
                    currRank, temp = Heuristic.calc3Kind(myHand, numCards)
                    if prevRank >= currRank:
                        probList.remove(maxChance)
                    else:
                        return [4, None, currRank, None]
                if generalProb[maxChance] == 5:
                    currRank, temp = Heuristic.calcStraight(myHand, numCards)
                    if prevRank >= currRank:
                        probList.remove(maxChance)
                    else:
                        return [5, None, currRank, None]
                if generalProb[maxChance] == 6:
                    currSuit, temp = Heuristic.calcFlush(myHand, numCards)
                    if prevSuit >= currSuit:
                        probList.remove(maxChance)
                    else:
                        return [6, currSuit, None, None]
                if generalProb[maxChance] == 7:
                    currRank, currRankTwo, temp = Heuristic.calcFullHouse(myHand, numCards)
                    if prevRank >= currRank:
                        if prevRankTwo >= currRankTwo:
                            probList.remove(maxChance)
                        else:
                            return [7, None, currRank, currRankTwo]
                    else:
                        return [7, None, currRank, currRankTwo]
                if generalProb[maxChance] == 8:
                    currRank, temp = Heuristic.calc4Kind(myHand, numCards)
                    if prevRank >= currRank:
                        probList.remove(maxChance)
                    else:
                        return [8, None, currRank, None]
                if generalProb[maxChance] == 9:
                    currSuit, currRank, temp = Heuristic.calcStraightFlush(myHand, numCards)
                    if prevRank >= currRank:
                        probList.remove(maxChance)
                    else:
                        return [9, currSuit, currRank, None]
            
            
            else:
                if generalProb[maxChance] == 2:
                    currRank, temp = Heuristic.calcPair(myHand, numCards)
                    return [2, None, currRank, None]
                if generalProb[maxChance] == 3:
                    currRank, currRankTwo, temp = Heuristic.calc2Pair(myHand, numCards)
                    return [3, None, currRank, currRankTwo]
                if generalProb[maxChance] == 4:
                    currRank, temp = Heuristic.calc3Kind(myHand, numCards)
                    return [4, None, currRank, None]
                if generalProb[maxChance] == 5:
                    currRank, temp = Heuristic.calcStraight(myHand, numCards)
                    return [5, None, currRank, None]
                if generalProb[maxChance] == 6:
                    currSuit, temp = Heuristic.calcFlush(myHand, numCards)
                    return [6, currSuit, None, None]
                if generalProb[maxChance] == 7:
                    currRank, currRankTwo, temp = Heuristic.calcFullHouse(myHand, numCards)
                    return [7, None, currRank, currRankTwo]
                if generalProb[maxChance] == 8:
                    currRank, temp = Heuristic.calc4Kind(myHand, numCards)
                    return [8, None, currRank, None]
                if generalProb[maxChance] == 9:
                    currSuit, currRank, temp = Heuristic.calcStraightFlush(myHand, numCards)
                    return [9, currSuit, currRank, None]
    
    def handCall(players,myHand,prevCall):
        if prevCall == [None]:
            #make a low call 
            handCall = AIone.guess(myHand,None,None,None,None,Game.pool_length)
            Game.prevCall = handCall
            return handCall 
        else: 
            #check probabilities of guess or bluff
            if prevCall[0] == 3 or prevCall[0] == 7:
                bluff_prob = AIone.bluff(myHand,prevCall[0],prevCall[2],prevCall[3],None,Game.pool_length)
            elif prevCall[0] == 6:
                bluff_prob = AIone.bluff(myHand,prevCall[0],None,None,prevCall[1],Game.pool_length)
            elif prevCall[0] == 9:
                bluff_prob = AIone.bluff(myHand,prevCall[0],prevCall[2],None,prevCall[1],Game.pool_length)
            else:
                bluff_prob = AIone.bluff(myHand,prevCall[0],prevCall[2],None,None,Game.pool_length)
            
            if bluff_prob == False:
                #make a hand call
                if prevCall[0] == 3 or prevCall[0] == 7:
                    handCall = AIone.guess(myHand,prevCall[0],prevCall[2],prevCall[3],None,Game.pool_length)
                elif prevCall[0] == 6:
                    handCall = AIone.guess(myHand,prevCall[0],None,None,prevCall[1],Game.pool_length)
                elif prevCall[0] == 9:
                    handCall = AIone.guess(myHand,prevCall[0],prevCall[2],None,prevCall[1],Game.pool_length)
                else:
                    handCall = AIone.guess(myHand,prevCall[0],prevCall[2],None,None,Game.pool_length)
                    
                Game.prevCall = handCall
                return handCall 
            else: 
                bluff = Game.bluffCall(players)
                return bluff

            
    

In [ ]:
Game.run()